# OSMnx

Testing the [OSMnx](https://geoffboeing.com/2016/11/osmnx-python-street-networks/) library for loading and processing OSM data.

See these [notebooks](https://github.com/gboeing/osmnx-examples/tree/master/notebooks) for examples of using the library.

In [ ]:
# !brew install spatialindex

In [48]:
import networkx as nx
import itertools
import osmnx as ox
import glob

%matplotlib inline
ox.config(log_console=True, use_cache=True, data_folder='')
ox.__version__

'0.10'

In [32]:
graphml_fnames = glob.glob('data/boeing_dataverse/06-CA-cities-street_networks-graphml/*.graphml')

In [44]:
G = ox.load_graphml(graphml_fnames[8])
len(G)

359

In [82]:
for e in G.edges:
    print(G.edges[e])

{'osmid': 25329497, 'oneway': True, 'lanes': '5', 'ref': 'I 405', 'name': 'San Diego Freeway', 'highway': 'motorway', 'maxspeed': '65 mph', 'length': 74.247}
{'osmid': 399710806, 'lanes': '7', 'ref': 'CA 1', 'name': 'Pacific Coast Highway', 'highway': 'primary', 'maxspeed': '35 mph', 'oneway': False, 'length': 48.692}
{'osmid': 56104363, 'lanes': '7', 'ref': 'CA 1', 'name': 'Pacific Coast Highway', 'highway': 'primary', 'maxspeed': '35 mph', 'oneway': False, 'length': 57.416999999999994, 'geometry': <shapely.geometry.linestring.LineString object at 0x1295505d0>}
{'osmid': [399706599, 56104364, 399706595, 399706607], 'lanes': ['5', '6', '4'], 'name': 'Redondo Avenue', 'highway': 'secondary', 'oneway': False, 'length': 210.94299999999998, 'geometry': <shapely.geometry.linestring.LineString object at 0x129550510>}
{'osmid': [379721185, 379721186, 399710829], 'lanes': ['7', '6'], 'ref': 'CA 1', 'name': 'Pacific Coast Highway', 'highway': 'primary', 'maxspeed': '35 mph', 'oneway': False, 'l